The following code is based on the program 'predict_video' by Felipe Tambasco (computervisioneng), licensed under AGPL-3.0.

Original source: https://github.com/computervisioneng/train-yolov8-custom-dataset-step-by-step-guide.

In [ ]:
# Initial settings

import locale
from google.colab import drive

locale.getpreferredencoding = lambda: 'UTF-8'

drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
# Define the folder where the tests will be conducted

ROOT_DIR = '/content/gdrive/MyDrive/SafeSkies/balloon_detection/'

In [ ]:
# Install the necessary libraries

!pip install ultralytics
!pip install opencv-python

In [ ]:
# Define the input and output files

import os
import cv2
from ultralytics import YOLO


VIDEOS_DIR = os.path.join(ROOT_DIR, 'balloon_videos')

video_path = os.path.join(VIDEOS_DIR, 'balloon_video_X.mp4')
video_path_out = os.path.join(VIDEOS_DIR, 'balloon_video_X_out.mp4')

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

model_path = os.path.join(ROOT_DIR, 'training_results', 'weights', 'best.pt')

In [ ]:
# Use the model to make predictions on each frame of the video

model = YOLO(model_path)

threshold = 0.5

while ret:

  results = model(frame)[0]

  for result in results.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = result

    if score > threshold:
      cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
      cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                  cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

  out.write(frame)
  ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()